In [2]:
import matplotlib.pyplot as plt

#Data Manipulation
import pandas as pd
import numpy as np

# Machine Learning
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import Input, Dense, Dropout
import tensorflow as tf

df = pd.read_csv("collected_data_180_step_4.csv")
    
print("From local")

df.head()

# Train test split
X_train_df, X_test_df = train_test_split(df, test_size=0.2, random_state=42)

# Train values
y1_train = np.array(X_train_df["Theta1"])
y2_train = np.array(X_train_df["Theta2"])
y3_train = np.array(X_train_df["Theta3"])

# Test values
y1_test = np.array(X_test_df["Theta1"])
y2_test = np.array(X_test_df["Theta2"])
y3_test = np.array(X_test_df["Theta3"])

# Remove the target values from the dataset
X_train_df = X_train_df.drop(["Theta1", "Theta2", "Theta3"], axis=1)
X_test_df = X_test_df.drop(["Theta1", "Theta2", "Theta3"], axis=1)

# Normalizing the dataset
X_train_norm = (X_train_df - X_train_df.mean()) / X_train_df.std()
X_test_norm = (X_test_df - X_test_df.mean()) / X_test_df.std()

# Convert to numpy arrays
X_train = X_train_norm.values
X_valid = X_test_norm.values

y1_train = y1_train.reshape(-1, 1) # Reshape y1_train to match X_train
y2_train = y2_train.reshape(-1, 1) # Reshape y2_train to match X_train
y3_train = y3_train.reshape(-1, 1) # Reshape y3_train to match X_train

y1_test = y1_test.reshape(-1, 1) # Reshape y1_train to match X_train
y2_test = y2_test.reshape(-1, 1) # Reshape y2_train to match X_train
y3_test = y3_test.reshape(-1, 1) # Reshape y2_train to match X_train

y_train = (y1_train, y2_train, y3_train)
y_valid = (y1_test, y2_test, y3_test)


From local


In [13]:
index_1 =  X_train_df.shape
index_1[1]

3

In [14]:
y_train

(array([[  4.],
        [136.],
        [ 60.],
        ...,
        [148.],
        [  0.],
        [ 28.]]),
 array([[108.],
        [132.],
        [144.],
        ...,
        [168.],
        [ 76.],
        [144.]]),
 array([[176.],
        [112.],
        [  8.],
        ...,
        [ 20.],
        [ 20.],
        [  0.]]))

<h2 style="font-size: 20px;">CREATING THE MODEL</h2>


In [15]:
def train_model(X_train, y_train, X_valid, y_valid, input_nodes, hidden_nodes, hidden_nodes_2, output_nodes, learning_rate, batch_size, epochs, dropout_rate):
    """
    Train a neural network model for multi-output regression.

    Args:
    X_train (numpy.ndarray): Input features for training.
    y_train (tuple of numpy.ndarray): Target labels for training.
    X_valid (numpy.ndarray): Input features for validation.
    y_valid (tuple of numpy.ndarray): Target labels for validation.
    input_nodes (int): Number of nodes in the input layers of the neural network.
    hidden_nodes (int): Number of nodes in the first hidden layer of the neural network.
    hidden_nodes_2 (int): Number of nodes in the second hidden layer of the neural network.
    output_nodes (tuple of int): Number of nodes in the output layers of the neural network.
    learning_rate (float): Learning rate for the optimizer.
    batch_size (int): Number of samples per batch during training.
    epochs (int): Number of epochs for training.
    dropout_rate (float): Dropout rate for regularization.

    Returns:
    tuple: A tuple containing:
        - nn_model (keras.Model): Trained neural network model.
        - history (dict): Training history.
    """
    # Unpack target labels
    y1_train, y2_train, y3_train = y_train
    y1_valid, y2_valid, y3_valid = y_valid

    # Define input layer
    input_layer = Input(shape=(input_nodes,))

    # Define dense layers
    dense_layer_1 = Dense(units=hidden_nodes, activation="relu")(input_layer)
    dropout_layer_1 = Dropout(dropout_rate)(dense_layer_1)  # Adding dropout after the first dense layer

    
    dense_layer_2 = Dense(units=hidden_nodes, activation="relu")(dropout_layer_1)
    dropout_layer_2 = Dropout(dropout_rate)(dense_layer_2)  # Adding dropout after the second dense layer

    dense_layer_3 = Dense(units=hidden_nodes_2, activation="relu")(dropout_layer_2)
    
    dense_layer_4 = Dense(units=hidden_nodes_2, activation="relu")(dense_layer_3)
    
    # Define Y1 output
    y1_output = Dense(units=1, activation="linear", name="y1_output")(dropout_layer_2)

    # Define Y2 output
    y2_output = Dense(units=1, activation="linear", name="y2_output")(dense_layer_3)

    # Define Y3 output
    y3_output = Dense(units=1, activation="linear", name="y3_output")(dense_layer_4)

    # Define the model with the input layer and a list of outputs
    model = Model(inputs=input_layer, outputs=[y1_output, y2_output, y3_output])

    # Specify the optimizer
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss={'y1_output': 'mse', 'y2_output': 'mse', 'y3_output': 'mse'},
                  metrics={'y1_output': tf.keras.metrics.RootMeanSquaredError(),
                           'y2_output': tf.keras.metrics.RootMeanSquaredError(),
                           'y3_output': tf.keras.metrics.RootMeanSquaredError()})

    # Train the model
    history = model.fit(X_train, {'y1_output': y1_train, 'y2_output': y2_train, 'y3_output': y3_train},
                        validation_data=(X_valid, {'y1_output': y1_valid, 'y2_output': y2_valid, 'y3_output': y3_valid}),
                        epochs=epochs, batch_size=batch_size, verbose=0)

    return model, history

import matplotlib.pyplot as plt

def plot_history(history):
    """
    Show the difference in loss and RMSE with each epoch of training the model.

    Args:
       history : Keras History object that stores information about the trained model.

    Returns:
        Plot showing the loss and RMSE with each epoch.
    """

    fig, axs = plt.subplots(3, 2, figsize=(20, 24))

    # Plot loss for y1
    axs[0, 0].plot(history.history['loss'], label='loss (y1)')
    axs[0, 0].plot(history.history['val_loss'], label='val_loss (y1)')
    axs[0, 0].set_xlabel('Epoch')
    axs[0, 0].set_ylabel('Loss')
    axs[0, 0].grid(True)
    axs[0, 0].legend()

    # Plot RMSE for y1
    axs[0, 1].plot(history.history['y1_output_root_mean_squared_error'], label='RMSE (y1)')
    axs[0, 1].plot(history.history['val_y1_output_root_mean_squared_error'], label='val_RMSE (y1)')
    axs[0, 1].set_xlabel('Epoch')
    axs[0, 1].set_ylabel('RMSE')
    axs[0, 1].grid(True)
    axs[0, 1].legend()

    # Plot loss for y2
    axs[1, 0].plot(history.history['loss'], label='loss (y2)')
    axs[1, 0].plot(history.history['val_loss'], label='val_loss (y2)')
    axs[1, 0].set_xlabel('Epoch')
    axs[1, 0].set_ylabel('Loss')
    axs[1, 0].grid(True)
    axs[1, 0].legend()

    # Plot RMSE for y2
    axs[1, 1].plot(history.history['y2_output_root_mean_squared_error'], label='RMSE (y2)')
    axs[1, 1].plot(history.history['val_y2_output_root_mean_squared_error'], label='val_RMSE (y2)')
    axs[1, 1].set_xlabel('Epoch')
    axs[1, 1].set_ylabel('RMSE')
    axs[1, 1].grid(True)
    axs[1, 1].legend()

    # Plot loss for y3
    axs[2, 0].plot(history.history['loss'], label='loss (y3)')
    axs[2, 0].plot(history.history['val_loss'], label='val_loss (y3)')
    axs[2, 0].set_xlabel('Epoch')
    axs[2, 0].set_ylabel('Loss')
    axs[2, 0].grid(True)
    axs[2, 0].legend()

    # Plot RMSE for y3
    axs[2, 1].plot(history.history['y3_output_root_mean_squared_error'], label='RMSE (y3)')
    axs[2, 1].plot(history.history['val_y3_output_root_mean_squared_error'], label='val_RMSE (y3)')
    axs[2, 1].set_xlabel('Epoch')
    axs[2, 1].set_ylabel('RMSE')
    axs[2, 1].grid(True)
    axs[2, 1].legend()

    plt.show()


<h2 style="font-size: 20px;">PARAMETERS OF THE MODEL</h2>

In [17]:
input_nodes = len(X_train[1])
output_nodes = 1
epochs = 100 #200/400 było git
hidden_nodes = 128  #60 było git
hidden_nodes_2 = 64  #64 było git
dropout_rate = 0.25  #0.25 było git
batch_size = 200
learining_rate = 0.001 #0.001 było git

<h2 style="font-size: 20px;">TESTETING THE MODEL</h2>

In [18]:
model, history = train_model(X_train, y_train, X_valid, y_valid, input_nodes, hidden_nodes, hidden_nodes_2, output_nodes, learining_rate, batch_size, epochs, dropout_rate)

KeyboardInterrupt: 

In [9]:
plot_history(history)

Y1_pred , Y2_pred, Y3_pred = model.predict(X_valid)
X_valid

570/570 [==============================] - 1s 2ms/step


array([[-1.95128227e-01, -8.16416779e-01, -9.07050459e-01],
       [ 2.69528669e+00, -2.38788082e-01, -1.14016130e+00],
       [-2.33704529e-01, -1.99668644e+00,  2.56798540e-01],
       ...,
       [-6.07289688e-02,  2.73701892e-01, -7.14374993e-01],
       [-6.45457975e-01, -1.22421086e-03, -1.68819738e+00],
       [-9.09219250e-02,  1.29539676e+00,  1.02117795e+00]])

In [10]:
plt.scatter(y1_test, Y1_pred)
combined_array1 = np.column_stack((y1_test, Y1_pred))


In [1]:
combined_array1.shape()

NameError: name 'combined_array1' is not defined

In [210]:
plt.scatter(y2_test, Y2_pred)
combined_array2 = np.column_stack((y2_test, Y2_pred))


In [191]:
combined_array2

array([[17.        , 13.73271561],
       [43.        , 41.54024887],
       [42.        , 40.99051666],
       ...,
       [12.        , 11.84365654],
       [41.        , 40.63547897],
       [12.        , 11.65812302]])

In [212]:
plt.scatter(y3_test, Y3_pred)
combined_array3 = np.column_stack((y3_test, Y3_pred))

In [192]:
combined_array3

array([[1.8       , 2.25056696],
       [9.8       , 9.72095966],
       [6.2       , 6.89110184],
       ...,
       [5.4       , 5.82195044],
       [9.4       , 9.263237  ],
       [8.8       , 9.11710453]])

In [205]:
def scale_this_point(point_in_space):
    X , Y , Z =   point_in_space
    X_norm = (X - X_train_df.loc[:,'X'].mean()) / X_train_df.loc[:,'X'].std()
    Y_norm = (Y - X_train_df.loc[:,'Y'].mean())/ X_train_df.loc[:,'Y'].std()
    Z_norm = (Z - X_train_df.loc[:,'Z'].mean()) / X_train_df.loc[:,'Z'].std()
    
    return X_norm, Y_norm, Z_norm

point = np.array([7.0197861,  3.27338002, 9.94331352])

point_in_space_norm = scale_this_point(point)
print(point_in_space_norm)
point_in_space_norm[0]

#prediction on the point
NEW_point_to_predict = np.array([[point_in_space_norm[0], point_in_space_norm[1],point_in_space_norm[2]]])
Theta1, Theta2, Theta3 = model.predict(NEW_point_to_predict)

(0.2493402757741605, 0.14454161491909812, 0.1868920259252381)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [206]:
print(Theta1)
print(Theta2)
print(Theta3)

[[24.71403]]
[[24.71293]]
[[26.515377]]


In [ ]:
model.save('D:/Mój dysk/PYTHON/MACHINE_LEARNING/LINEAR_REGRESSION/INVERSE_KINEMATICS_NEURAL_NETWORK/MODEL/my_model2.keras')